# Boston_House_dataset

In [4]:
import pandas as pd
import numpy as np
from sklearn import datasets

In [2]:
df_ = pd.read_csv("c:/python/boston_house.csv")
df = df_.iloc[:,1:] #index 행 삭제

# 작업형 1 - 데이터 대치

In [3]:
# df_ = pd.read_csv("c:/python/boston_house.csv")

sort_df = df.sort_values(by='CRIM', ascending=False)

#값 치환
sort_df['CRIM'].iloc[0:10] = sort_df['CRIM'].iloc[9]

#평균 (2개중 하나로 풀이)
#sort_df.loc[sort_df['AGE'] >= 80, 'CRIM'].mean()
mean_value = sort_df[sort_df['AGE'] >= 80]['CRIM'].mean()

print(mean_value)

5.759386624999999


# 작업형 1 - 이상치 확인 후 합계

In [27]:
sort_df = df['RM'] # 임의의 데이터를 활용 부여

mean_value = sort_df.mean()
std_value = sort_df.std()

# 1분위수 , 4분위수
lower = mean_value - (1.5 * std_value)
upper = mean_value + (1.5 * std_value)

print(sort_df[sort_df<lower].sum() + sort_df[sort_df>upper].sum())

380.602


# 작업형 1 - 결측치 치환

In [38]:
dt = pd.read_csv("./basic1.csv")

# 기존 확인
std = dt['f1'].std()
mean = dt['f1'].mean()
print(std)

# 결측치 보간
dt_sort = dt['f1'].fillna(mean)
print(dt_sort.std())

19.453893291430727
16.122908466726415


# 임의 데이터 활용
- 작업형 1 - Scaler 사용

In [5]:
# 패키지 help 관련 ~ .__all__
import sklearn.preprocessing
print(sklearn.preprocessing.__all__)

['Binarizer', 'FunctionTransformer', 'KBinsDiscretizer', 'KernelCenterer', 'LabelBinarizer', 'LabelEncoder', 'MultiLabelBinarizer', 'MinMaxScaler', 'MaxAbsScaler', 'QuantileTransformer', 'Normalizer', 'OneHotEncoder', 'OrdinalEncoder', 'PowerTransformer', 'RobustScaler', 'SplineTransformer', 'StandardScaler', 'add_dummy_feature', 'PolynomialFeatures', 'binarize', 'normalize', 'scale', 'robust_scale', 'maxabs_scale', 'minmax_scale', 'label_binarize', 'quantile_transform', 'power_transform']


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

dt = pd.read_csv("./basic1.csv")

In [49]:
stand_scaler = StandardScaler()
minmax = MinMaxScaler()

mm_scale = minmax.fit_transform(dt[['f5']])
std_scale = stand_scaler.fit_transform(dt[['f5']])


# MinMaxScaler ~ 특정 조건 이상인 데이터 갯수 출력
print(len(mm_scale[mm_scale > 0.80]))

# 작업형2
## E-Commerce Dataset (분류)


In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
# import xgboost as XGB

In [6]:
dt = pd.read_csv("C:/python/data_zip/ecommerce.csv")

# 데이터치환
mos = pd.get_dummies(dt['Mode_of_Shipment'])
dt['Gender'].replace(["M","F"], ['0','1'], inplace=True)

# 데이터 임의의 이상치 설정 후 제거 (평균 +- (1.5*표준편차))
lower = dt['Weight_in_gms'].mean() - (1.5 * dt['Weight_in_gms'].std())
upper = dt['Weight_in_gms'].mean() + (1.5 * dt['Weight_in_gms'].std())

dt[(dt['Weight_in_gms'] > lower) & (dt['Weight_in_gms'] < upper)]

## One-hot Encoding 활용 + Scaler 추가

In [31]:
dt_crop = dt[(dt['Weight_in_gms'] > lower) & (dt['Weight_in_gms'] < upper)]

crop = dt_crop[['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product',
                'Discount_offered', 'Weight_in_gms']]

crop_all = pd.concat([mos, crop, dt['Reached.on.Time_Y.N']], axis=1).dropna()

scaler = StandardScaler()
crop_all['Cost_of_the_Product'] = scaler.fit_transform(crop_all[['Cost_of_the_Product']])
crop_all['Weight_in_gms'] = scaler.fit_transform(crop_all[['Weight_in_gms']])

# Train, Test 데이터 분할
train, test = train_test_split(crop_all, test_size=0.2, random_state=23)

# X,y 설정
x_train, y_train = train.iloc[:,0:8], train.iloc[:,8]
x_test, y_test = test.iloc[:,0:8], test.iloc[:,8]

# 모델(분류기)
rf = RandomForestClassifier(random_state=23)
ada = AdaBoostClassifier(random_state=23)
gbm = GradientBoostingClassifier(random_state=23)
dtt = DecisionTreeClassifier(random_state=23)

#학습
rf.fit(x_train, y_train)
ada.fit(x_train, y_train)
gbm.fit(x_train, y_train)
dtt.fit(x_train, y_train)

#예측 (분류기>> predict_proba)
y_pred = rf.predict_proba(x_test)[:,1]
y_pred_ = ada.predict_proba(x_test)[:,1]
y_pred__ = gbm.predict_proba(x_test)[:,1]
y_pred___ = dtt.predict_proba(x_test)[:,1]

from sklearn.metrics import roc_auc_score
print("RandomForest ROC_AUC : {}".format(roc_auc_score(y_test, y_pred)))
print("Adaboost ROC_AUC : {}".format(roc_auc_score(y_test, y_pred_)))
print("GradientBoosting ROC_AUC : {}".format(roc_auc_score(y_test, y_pred__)))
print("DecisionTree ROC_AUC : {}".format(roc_auc_score(y_test, y_pred___)))

RandomForest ROC_AUC : 0.750172041037522
Adaboost ROC_AUC : 0.7336527561929995
GradientBoosting ROC_AUC : 0.746258585325664
DecisionTree ROC_AUC : 0.6247369205801224


## 숫자 데이터 Only 활용

In [26]:
# 숫자 데이터인 열의 이름 확인 후 데이터 Crop
crop = dt[dt.describe(include='int64').columns].iloc[:,1:]

train, test = train_test_split(crop, test_size=0.2, shuffle=True ,random_state=23)

x_train, y_train = train.iloc[:,0:6], train.iloc[:,6]
x_test, y_test = test.iloc[:,0:6], test.iloc[:,6]

rf = RandomForestClassifier(random_state=23)
ada = AdaBoostClassifier(random_state=23)
gbm = GradientBoostingClassifier(random_state=23)
dtt = DecisionTreeClassifier(random_state=23)

rf.fit(x_train, y_train)
ada.fit(x_train, y_train)
gbm.fit(x_train, y_train)
dtt.fit(x_train, y_train)

rf_pred = rf.predict_proba(x_test)[:,1]
ada_pred = ada.predict_proba(x_test)[:,1]
gbm_pred = gbm.predict_proba(x_test)[:,1]
dtt_pred = dtt.predict_proba(x_test)[:,1]

from sklearn.metrics import roc_auc_score
print("RandomForest ROC_AUC : {}".format(roc_auc_score(y_test, rf_pred)))
print("Adaboost ROC_AUC : {}".format(roc_auc_score(y_test, ada_pred)))
print("GradientBoosting ROC_AUC : {}".format(roc_auc_score(y_test, gbm_pred)))
print("DecisionTree ROC_AUC : {}".format(roc_auc_score(y_test, dtt_pred)))

RandomForest ROC_AUC : 0.7358787577104264
Adaboost ROC_AUC : 0.741801203562106
GradientBoosting ROC_AUC : 0.7454135149282071
DecisionTree ROC_AUC : 0.6188543238259454
